In [106]:
from datetime import datetime
import sys
import time
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
from PIL import Image
import html
from markdownify import markdownify 
import folium




endpoint_url = "https://computational-publishing-service.wikibase.cloud/query/sparql"

endpoint_url = "https://query.wikidata.org/sparql"

query = """
# Get images of brutalist buildings
SELECT DISTINCT ?item ?itemLabel ?image ?placeLabel ?countryLabel (YEAR(?inception) AS ?year) ?latitude ?longitude ?itemDescription WHERE {
  ?item wdt:P31 wd:Q41176;
    wdt:P149 wd:Q994776;
    wdt:P625 ?coordinates;
    wdt:P571 ?inception;
    wdt:P131 ?place;
    wdt:P17 ?country;
    wdt:P18 ?image.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,[AUTO_LANGUAGE]". }
  BIND(geof:latitude(?coordinates) AS ?latitude).
  BIND(geof:longitude(?coordinates) AS ?longitude).
}
"""


locations = []

def run_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setMethod("POST") #this NEEDS to be added to get results (not included in the wikibase python example code)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def get_img():

    print('<script src="https://unpkg.com/masonry-layout@4/dist/masonry.pkgd.min.js"></script>')
    print('<script src="https://unpkg.com/imagesloaded@4/imagesloaded.pkgd.min.js"></script>')
    print("<link rel='stylesheet' href='./masonrydesign.css'>")
    print('<div class="img_grid">')
    q = query

    results = run_query(endpoint_url, q)

    for item in results["results"]["bindings"]:    

      alt='image of a ' + item['itemDescription']['value'] if 'itemDescription' in item else 'image of a brutalist building'
      title=item['itemLabel']['value'] if 'itemLabel' in item else ''
      year=item['year']['value'] if 'year' in item else ''
      latitude=item['latitude']['value'] if 'latitude' in item else ''
      longitude=item['longitude']['value'] if 'longitude' in item else ''
      place=item['placeLabel']['value'] + ', ' + item['countryLabel']['value'] 
      item_url=item['item']['value'] if 'item' in item else ""
      image_url=item['image']['value'] if 'image' in item else ''

      locations.append((title, latitude, longitude, image_url, item_url, place, alt))







In [107]:
get_img()

m = folium.Map(location=[41, -23.8], zoom_start=2)

for name, lat, lon, url, item, place, alt in locations:
    folium.Marker(
        location=[lat, lon],
        popup=f"""<h3><a href='{item}'>{name}</a></h3>
        {place}
        <img src='{url}' alt='{alt}' style='width:200px;height:200px;object-fit:contain;'>
        <a href='{url}'>Full size image</a>""",  # Clickable label
        tooltip=name  # Hover label
    ).add_to(m)
    

m



<script src="https://unpkg.com/masonry-layout@4/dist/masonry.pkgd.min.js"></script>
<script src="https://unpkg.com/imagesloaded@4/imagesloaded.pkgd.min.js"></script>
<link rel='stylesheet' href='./masonrydesign.css'>
<div class="img_grid">


In [108]:
#get_graph()